In [1]:
import sys
sys.path.append('/home/kyungtak.lim/gbspy')


import gbspy as g
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py

import multiprocessing as mp
from matplotlib.animation import FuncAnimation
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from matplotlib.ticker import AutoMinorLocator
import matplotlib.lines as mlines
from matplotlib.ticker import AutoMinorLocator

# Get the current working directory|
cwd = os.getcwd()

In [2]:
import numpy as np
from pp import Sim

ModuleNotFoundError: No module named 'pp'

In [4]:
def plot3d(sim:Sim, data, phi1=None, phi2=None, cmap=None, lights=None, color_psi=None,
           vmax=None, vmin=None, phi_in=None, rho_in=None, vmin_in=None, vmax_in=None,
           window_size_plot=[1200, 900], zoom_val = None):
    """
    Plot the 3D tokamak representation of data, between phi1 and phi2.

    NB: It returns the pyvista.plotter object, allowing to manipulate the orientation, zoom, etc
        and then save it.
        TO CLOSE properly the object use plotter.close() .

    Parameters
    ------------
    gbssim : gbspy.pp.Sim
        Simulation object
    data : np.array 3D,
        Data to plot
    phi1, phi2: float, optional
        Toroidal angle as plot limits
    cmap: str, optional
        Colormap name
    lights: list of pyvista.Light, optional
        list of pyvista.Light objects used in the plot
        NB: default light are thought for default cmap (hot),
            to have no light use: lights = []
            for different light add custom pyvista.Light
    color_psi: str, optional
        color for the separatrix line
    vmin, vmax: float, optional
        Vmin and Vmax for the color plots
    phi_in, rho_in: float, optional
        max angle and rho_psi value to plot an internal surface (0 < rho < 1),
        from phi2 to phi_in.
        If only phi_in is given, psi_in is set to a defulat surface in the core.
    vmin_in, vmax_in: float, optional
        as vmin,vmax but for the inner surface
        NB: The default values for the cmap limits are different for the tokamak plot
            and for the inner surface
    window_size_plot: [x,y] floats, optional
        the size of the window
    zoom_val: float, optional
        value for plotter.camera.zoom, typical good = 1.75
    Returns
    -------
    plotter: pyvista.Plotter,
        plotter where the 3D plot is drawn on,
        possibility to zoom in with plotter.camera.zoom(number>1),
        to save with plotter.save_graphic(filename), etc
    """

    if phi1 is None:
        phi1 = 0
    if phi2 is None:
        phi2 = 4
    if phi2 <= (phi1+sim.dz/2):
        raise ValueError("phi2 has to be larger than phi1+dz/2")

    if cmap is None:
        cmap = 'turbo'

    if color_psi is None:
        color_psi = 'w'

    op_val = 1. #Opacity value
    if vmax is None:
        vmax=np.max(data)
    if vmin is None:
        vmin=np.min(data)

    # Create a plotter
    plotter = pv.Plotter(window_size=window_size_plot)
    plotter.set_background("white")

    iphi1 = np.argmin(np.abs(sim.z-phi1))
    phi1=sim.z[iphi1]
    Z, R = np.meshgrid(sim.y[2:-2], sim.x[2:-2])  # radial and vertical coords
    X1 = (R+sim.attribute['rorho_s']-0.5*(sim.x[2]+sim.x[-2])) * np.cos(phi1)
    Y1 = (R+sim.attribute['rorho_s']-0.5*(sim.x[2]+sim.x[-2])) * np.sin(phi1)
    Z1 = Z
    data1 = data[2:-2,2:-2,iphi1]
    data1 = data1.flatten()

    # --- Surface plot 1 (axial cut at phi1)
    grid1 = pv.StructuredGrid(X1, Y1, Z1)
    grid1["values"] = data1
    plotter.add_mesh(grid1, scalars="values", cmap=cmap, clim=[vmin,vmax],
                     opacity=op_val, show_scalar_bar=False, name="plane_phi1")

    grid1["mask"] = sim.Psi[2:-2,2:-2].flatten()
    iso_value = sim.Psi[sim.iyxpt,sim.ixxpt]
    contour_line = grid1.contour(isosurfaces=[iso_value], scalars="mask")
    plotter.add_mesh(contour_line, color=color_psi, line_width=3, name="sep_phi1")

    # --- Surface plot 2 (axial cut at phi2)
    iphi2 = np.argmin(np.abs(sim.z-phi2))
    phi2=sim.z[iphi2]
    X2 = (R+sim.attribute['rorho_s']-0.5*(sim.x[2]+sim.x[-2])) * np.cos(phi2)
    Y2 = (R+sim.attribute['rorho_s']-0.5*(sim.x[2]+sim.x[-2])) * np.sin(phi2)
    Z2 = Z
    data2 = data[2:-2,2:-2,iphi2]
    data2 = data2.flatten()
    grid2 = pv.StructuredGrid(X2, Y2, Z2)
    grid2["values"] = data2
    plotter.add_mesh(grid2, scalars="values", cmap=cmap, clim=[vmin,vmax],
                     opacity=op_val, show_scalar_bar=False, name="plane_phi2")

    grid2["mask"] = sim.Psi[2:-2,2:-2].flatten()
    contour_line = grid2.contour(isosurfaces=[iso_value], scalars="mask")
    plotter.add_mesh(contour_line, color=color_psi, line_width=3, name="sep_phi2")


    # --- 2D flux surface bottom
    R, Phi = np.meshgrid(sim.x[2:-2], sim.z[iphi1:iphi2+1])
    X3 = (R+sim.attribute['rorho_s']-0.5*(sim.x[2]+sim.x[-2])) * np.cos(Phi)
    Y3 = (R+sim.attribute['rorho_s']-0.5*(sim.x[2]+sim.x[-2])) * np.sin(Phi)
    Z3 = np.ones_like(R)*sim.y[2]
    data3 = data[2,2:-2,iphi1:iphi2+1]
    data3 = data3.flatten()
    grid3 = pv.StructuredGrid(X3, Y3, Z3)
    grid3["values"] = data3
    plotter.add_mesh(grid3, scalars="values", cmap=cmap, clim=[vmin,vmax],
                     opacity=op_val, show_scalar_bar=False, smooth_shading=True,
                     name="bottom_surf")


    # --- 2D flux surface top
    X4 = (R+sim.attribute['rorho_s']-0.5*(sim.x[2]+sim.x[-2])) * np.cos(Phi)
    Y4 = (R+sim.attribute['rorho_s']-0.5*(sim.x[2]+sim.x[-2])) * np.sin(Phi)
    Z4 = np.ones_like(R)*sim.y[-3]
    data4 = data[-3,2:-2,iphi1:iphi2+1]
    data4 = data4.flatten()
    grid4 = pv.StructuredGrid(X4, Y4, Z4)
    grid4["values"] = data4
    plotter.add_mesh(grid4, scalars="values", cmap=cmap, clim=[vmin,vmax],
                     opacity=op_val, show_scalar_bar=False, smooth_shading=True,
                     name="top_surf")

    # --- 2D flux surface inner
    Z, Phi = np.meshgrid(sim.y[2:-2], sim.z[iphi1:iphi2+1])
    R = np.zeros_like(Z)
    X5 = (R+sim.attribute['rorho_s']-0.5*(sim.x[2]+sim.x[-2])) * np.cos(Phi)
    Y5 = (R+sim.attribute['rorho_s']-0.5*(sim.x[2]+sim.x[-2])) * np.sin(Phi)
    Z5 = Z
    data5 = data[2:-2,2,iphi1:iphi2+1]
    data5 = data5.flatten()
    grid5 = pv.StructuredGrid(X5, Y5, Z5)
    grid5["values"] = data5
    plotter.add_mesh(grid5, scalars="values", cmap=cmap, clim=[vmin,vmax],
                     opacity=op_val, show_scalar_bar=False, smooth_shading=True,
                     name="inner_surf")

    # --- 2D flux surface outer
    R = np.ones_like(Z)*sim.x[-3]
    X6 = (R+sim.attribute['rorho_s']-0.5*(sim.x[2]+sim.x[-2])) * np.cos(Phi)
    Y6 = (R+sim.attribute['rorho_s']-0.5*(sim.x[2]+sim.x[-2])) * np.sin(Phi)
    Z6 = Z
    data6 = data[2:-2,-3,iphi1:iphi2+1]
    data6 = data6.flatten()
    grid6 = pv.StructuredGrid(X6, Y6, Z6)
    grid6["values"] = data6
    plotter.add_mesh(grid6, scalars="values", cmap=cmap, clim=[vmin,vmax],
                     opacity=op_val, show_scalar_bar=False,smooth_shading=True,
                     name="outer_surf")


    #Inside Flux surface
    if phi_in:
        if phi_in <= (phi2+sim.dz/2):
            raise ValueError("phi_in has to be larger than phi2+dz/2")
        rho_vals = (sim.Psi[sim.iy0,sim.ix0:]-sim.Psi[sim.iy0,sim.ix0])/(sim.Psi[sim.iyxpt,sim.ixxpt]-sim.Psi[sim.iy0,sim.ix0])
        if rho_in is None:
            rho_in = 0.85
        ixpsi_in = np.argmin(np.abs(rho_vals-rho_in))+sim.ix0
        iso_value = sim.Psi[sim.iy0,ixpsi_in]
        iphi3 = np.argmin(np.abs(sim.z-phi_in))
        R, Z, Phi = np.meshgrid(sim.x[2:-2], sim.y[2:-2], sim.z[iphi2:iphi3+1])
        X_in = (R+sim.attribute['rorho_s']-0.5*(sim.x[2]+sim.x[-2])) * np.cos(Phi)
        Y_in = (R+sim.attribute['rorho_s']-0.5*(sim.x[2]+sim.x[-2])) * np.sin(Phi)
        Z_in = Z
        psi2D = sim.Psi[2:-2,2:-2]
        psi3D = np.repeat(psi2D[:,:,np.newaxis],(iphi3-iphi2+1),axis=2)
        psi3D[Z<sim.yxpt] = 0.
        data_in = data[2:-2,2:-2,iphi2:iphi3+1]
        grid_in = pv.StructuredGrid(X_in,Y_in,Z_in)
        grid_in["Psi"] = psi3D.T.flatten()
        grid_in["values"] = data_in.T.flatten()
        surface_in = grid_in.contour(isosurfaces=[iso_value], scalars="Psi")
        if vmin_in is None:
            vmin_in = np.min(surface_in['values'])
        if vmax_in is None:
            vmax_in = np.max(surface_in['values'])
        plotter.add_mesh(surface_in, scalars="values", cmap=cmap, clim=[vmin_in,vmax_in],
                         show_scalar_bar=False,smooth_shading=True,name="psi_in_surf")


    # --- Camera and lighting
    plotter.view_vector((0, -1, 0.3))

    if lights is None:
        top_light = pv.Light(position=(0, 0, 1), intensity=0.5)
        plotter.add_light(top_light)

        front_light = pv.Light(light_type='headlight',intensity=0.5)
        plotter.add_light(front_light)
    else:
        for light in lights:
            plotter.add_light(light)

    if zoom_val is not None:
        plotter.camera.zoom(zoom_val)

    plotter.show(auto_close=False)
    return plotter


NameError: name 'Sim' is not defined